# Load modules

In [1]:
import numpy as np
import pandas as pd

import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib notebook

# Load data

In [2]:
data = np.load(os.path.join('data', 'preprocessed', 'preprocessed_dataset_1.npz'), allow_pickle = True)

#training dataset 
x_train = data['x_train']
y_train = data['y_train']
idx_train = data['idx_train']

#test dataset 
x_test = data['x_test']
idx_test = data['idx_test']


# Do feature extraction magic

In [3]:
# TBD

# Save feature extracted dataset

In [ ]:
#Save
np.savez(os.path.join('data', 'feature_extracted', 'feature_extracted_dataset_1'), 
         x_train =x_train,
         y_train = y_train,
         idx_train = idx_train,
         x_test = x_test,
         idx_test = idx_test)